# Function Approximation


Sciences Po, Spring 2018


## Outline

1. Overview of Approximation Methods
	1. Interpolation
	1. Regression
1. Polynomial Interpolation
1. Spline Interpolation
1. Multidimensional Approximation




## Approximation Methods

* Confronted with a non-analytic function $f$ (i.e. something not like $log(x)$), we need a way to numerically represent $f$ in a computer.
	* If your problem is to compute a value function in a dynamic problem, you don't have an *analytic representation* of $V$.
	* If you need to compute an equilibrium distribution for your model, you probably can't tell it's from one parametric family or another.
* Approximations use *data* of some kind which informs us about $f$. Most commonly, we know the function values $f(x_i)$ at a corresponding finite set of points $X = \{x_i\}_{i=1}^N$.
* The task of approximation is to take that data and tell us what the function value is at $f(y),y\not \in X$.
* To an economist this should sound very familiar: take a dataset, learn it's structure, and make predictions.
* The only difference is that we can do much better here, because we have more degree's of freedom (we can choose our $X$ in $Y=\beta X + \epsilon$)


## Some Taxonomy

* Local Approximations: approximate function and it's derivative $f,f'$ at a *single* point $x_0$. Taylor Series:
	$$ f(x) = f(x_0) + (x-x_0)f'(x_0) + \frac{(x-x_0)^2}{2}f''(x_0) + \dots + \frac{(x-x_0)^n}{n!}f^{n}(x_0) $$
* Interpolation or *Colocation*: find a function $\hat{f}$ that is a good fit to $f$, and require that $\hat{f}$ *passes through* the points. If we think of there being a *residual* $\epsilon_i = f(x_i) - \hat{f}(x_i)$ at each grid point $i$, this methods succeeds in setting $\epsilon_i=0,\forall i$.
* Regression: Minimize some notion of distance (or squared distance) between $\hat{f}$ and $f$, without the requirement of pass through. 

## Doing Interpolation in Julia

* In practice, you will make heavy use of high-quality interpolation packages in julia.
* List in the end.
* Nevertheless, function approximation is *extremely* problem-specific, so sometimes a certain approach does not work for your problem.
* This is why we will go through the mechanics of some common methods.
* I would like you to know where to start drilling if you need to go and hack somebody elses code.


## Interpolation Basics
* Let $f$ be a smooth function mapping $\mathbb{R}^d \mapsto \mathbb{R}$, and define $\hat{f}(\cdot;c)$ to be our parametric approximation function. We generically define this as

$$  
    \hat{f}(x;c) = \sum_{j=1}^J c_j \phi_j(x) 
$$

where 
* $\phi_j : \mathbb{R}^d \mapsto \mathbb{R}$ is called a **basis function**,
* $c={c_1,c_2,\dots,c_J}$ is a coefficient vector
* The integer $J$ is the *order* of the interpolation.
* Our problem is to choose $(\phi_i,c)$ in some way.

We will construct a *grid* of $M\geq J$ points ${x_1,\dots,x_M}$ within the domain $\mathbb{R}^d$, and we will denote the *residuals* at each grid point by $\epsilon = {\epsilon_1,\dots,\epsilon_M}$:	

$$ 
    \left[\begin{array}{c}
        \epsilon_1 \\
         \vdots \\
        \epsilon_M \\ \end{array} \right]  = \left[\begin{array}{c} f(x_1) \\ \vdots \\ f(x_M)  \end{array} \right] - \left[\begin{array}{ccc} 
        \phi_1(x_1) & \dots & \phi_J(x_1) \\   
        \vdots      & \ddots & \vdots \\   
        \phi_1(x_M) & \dots & \phi_J(x_M)    
        \end{array} \right]  \cdot 
        \left[\begin{array}{c} c_1 \\ \vdots \\ c_J  \end{array} \right]
$$

* *Interpolation* or *colocation* occurs when $J=M$, i.e. we have a square matrix of basis functions, and can exactly solve this.
* We basically need to solve the system

$$ 
\begin{aligned} 
\sum_{j=1}^n c_j \phi_j(x_i) &= f(x_i),\forall i=1,2,\dots,n \\
                  \mathbf{\Phi c}&= \mathbf{y}
\end{aligned}
$$

where the second line uses vector notation, and $\mathbf{y}$ has all values of $f$.
* Solution: $\mathbf{c}= \mathbf{\Phi}^{-1}y$.




## Regression Basics

* Clearly, on the previous slide we required that there are as many interpolation nodes as there are basis functions - we had $J$ equations for $M$ unknowns, so there exists a unique solution for $c$.
* We needed to *invert* $\mathbf{\Phi}$.
* If we have more, $M>J$ say, interpolation nodes than basis functions, we cannot do that. Instead we can define a loss function, and minimize it. 
* In the case of squared loss, of course, this leads to the least squares solution:

$$ 
\begin{aligned} e_i &= f(x_i) - \sum_{j=1}^n c_j \phi_j(x_i) \\
        \min_c e_i^2 & \implies \\
        c            &= (\Phi'\Phi)^{-1} \Phi'y
\end{aligned}
 $$


## Spectral and Finite Element Methods

* **Spectral Methods** are such that the basis functions are non-zero over the entire domain of $f$.
	* Polynomial interpolation
	* Chebychev interpolation
* **Finite Element** methods are such that basis functions are non-zero only on a subset of the domain.
	* Splines
		* Linear splines, i.e. splines of degree 1, a.k.a. *linear approximation*
		* Higher order splines, mainly the *cubic spline*.


## What makes a **good** Approximation?

* Should be arbitrarily accurate as we increase $n$.
* $\Phi$ Should be efficiently (fast) computable. If $\Phi$ were differentiable, we could easily get e.g. $\hat{f}'(x) = \sum_{j=1}^J c_j \phi_j'(x_i)$
* $c$ Should be efficiently (fast) computable.



# Polynomial Interpolation

* For any continuous real-valued function $f$ on interval $[a,b]$, and an $\epsilon>0$, there is a polynomial $p$, such that
	$$ || f - p ||_\infty \equiv \sup_{x\in[a,b]} |f(x)-p(x)| < \epsilon $$
* However, the choice of $p$ is critical. *Orthogonal Polynomials* have been shown to perform very well.

> <span style="color:red">Definition - Orthogonal Polynomials</span>: an orthogonal polynomial sequence is a family of polynomials such that any two different polynomials in the sequence are orthogonal to each other under some inner product. 

* There are many families that satisfy this. See [@judd-book] <cite data-cite=juddbook></cite> table 6.3 for an overview.
* We will now look at a widely used family, the **Chebyshev polynomial**.


## Chebyshev Nodes

* *Chebyshev Nodes* are defined in the interval $[-1,1]$ as 
    $$ x_i = \cos\left(\frac{2k-1}{2n} \pi\right), k=1,\dots,n $$
* Which maps to general interval $[a,b]$ as
	$$ x_i = \frac{1}{2} (a+b) + \frac{1}{2} (b-a) \cos\left(\frac{2k-1}{2n} \pi\right) , k=1,\dots,n $$
* Chebyshev nodes are **not** evenly spaced: there are more points towards the boundaries. You might remember that from Gaussian Integration nodes.

In [1]:
using Plots
using FastGaussQuadrature
nodes = gausschebyshev(21)  # generate 11 Chebyshev Nodes
gr()
scatter(nodes,ones(21),ylims=(0.9,1.1),m=(:red,:+),legend=false,size=(600,100),yticks=nothing)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0


## What Polynomial to use? What form for $\Phi$?

* In principle the *monomial basis* could be used. It is just the power functions of $x$: $1,x,x^2,x^3,\dots$
* Stacking this up for each evaluation node (Chebyshev, or any other), gives the *Vandermonde Matrix*:
  
$$ 
V = \left[\begin{matrix} 
        1 & x_1 & \dots & x_1^{n-2} & x_1^{n-1} \\ 
        1 & x_2 & \dots & x_2^{n-2} & x_2^{n-1} \\ 
        \vdots & \vdots & \ddots &  & \vdots \\ 
        1 & x_m & \dots & x_m^{n-2} & x_m^{n-1} 
        \end{matrix} \right]
$$


for the case with $m$ evaluation nodes for $x$, and $n$ basis functions for each $x_i$.
* $V$ is often ill-conditioned and thus a bad choice.
* A much better polynomial basis is - *surprise, surprise* - the *Chebyshev Polynomial basis*.



## Evaluating the Chebyshev Polynomial and Basis Function
* As before, this Basis is defined in $[-1,1]$, so for general $x\in[a,b]$ we normalize $x$ to
	$$ z_i = 2\frac{x_i-a}{b-a} -1 $$
* There are several ways to obtain the value of the order $j$ Chebyshev polynomial at $z$, e.g. to get $\sum_{i=0}^n
c_i T_i(z)$.
	1. use definition $T_j(z) = \cos(\arccos(z)j)$. Or
	2. Recursively we have that $\phi_j(x) = T_{j-1}(z)$, and
            
$$
    \begin{aligned}
    T_0(z) =& 1 \\
    T_1(z) =& z \\
    T_{i+1}(z) =& 2zT_i(z) - T_{i-1}(z),i=1,\dots,n
    \end{aligned}
$$

### Constructing $\Phi$ as $T$ evaluated *at* the Chebyshev Nodes
* Combining Chebyshev nodes evaluated at the roots $T$ of the Cheby polynomial to construct $\Phi$ is a particularly good idea.
* Doing so, we obtain an interpolation matrix $\Phi$ with typical element
	$$ \phi_{ij} = \cos\left( \frac{(n-i+0.5)(j-1)\pi}{n}\right)  $$
* And we obtain that $\Phi$ is indeed orthogonal
	$$ \Phi^T \Phi = \text{diag}\{n,n/2,n/2,\dots,n/2\}  $$




## (Chebyshev) Interpolation Proceedure

* Let's summarize this proceedure.
* Instead of Chebyshev polynomials we could be using any other suitable family of polynomials.
* To obtain a Polynomial interpolant $\hat{f}$, we need:
	1. a function to $f$ interpolate. We need to be able to get the function values somehow.
	1. A set of (Chebyshev) interpolation nodes at which to compute $f$
	1. An interpolation matrix $\Phi$ that corresponds to the nodes we have chosen.
	1. A resulting coefficient vector $c$
* To obtain the value of the interpolation at $x'$ off our grid, we also need a way to evaluate $\Phi(x')$	.
	1. Evaluate the Basis function $\Phi$ at $x'$: get $\Phi(x')$
	2. obtain new values as $y = \Phi(x') c$.

## Polynomial Interpolation with `Julia`: `ApproxFun.jl`

* [`ApproxFun.jl`](https://github.com/ApproxFun/ApproxFun.jl) is a Julia package based on the Matlab package [`chebfun`](http://www.chebfun.org). It is quite amazing.
* More than just *function approximation*. This is a toolbox to actually *work* with functions.
* given 2 functions $f,g$, we can do algebra with them, i.e. $h(x) = f(x) + g(x)^2$
* We can differentiate and integrate
* Solve ODE's and PDE's
* represent periodic functions
* Head over to the [website and look at the readme](https://github.com/JuliaApproximation/ApproxFun.jl).

In [2]:
# works even with discontinuities!
using ApproxFun
ff = x->sign(x-0.1)/2 + cos(4*x);  # sign introduces a jump at 0.1
x  = Fun(identity)	# set up a function space
space(x)
f  = ff(x)	# define ff on that space
plot(f)	# plot

INFO: Precompiling module ApproxFun.
Expr(:call, Expr(:., :Base, :include_from_node1)::Any, "/Users/74097/.julia/v0.6/FFTW/src/FFTW.jl")::Any
  ** incremental compilation may be broken for this module **

Expr(:call, Expr(:., :Base, :include_from_node1)::Any, "/Users/74097/.julia/v0.6/FFTW/src/FFTW.jl")::Any
  ** incremental compilation may be broken for this module **



<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.75 
 
 
 -0.50 
 
 
 -0.25 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
<polyline clip-path="url(#clip2102)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 336.772,88.2731 336.644,88.1605 336.388,87.9368 336.003,87.6054 335.491,87.171 334.852,86.6401 334.086,86.0208 333.194,85.3228 332.177,84.5573 331.036,83.7372 
 329.772,82.8767 328.385,81.9919 326.877,81.1001 325.249,80.22 323.503,79.3718 321.64,78.5769 319.662,77.8579 317.571,77.2382 315.368,76.7426 313.054,76.3963 
 310.634,76.2251 308.107,76.2553 305.477,76.5134 302.745,77.0257 299.914,77.8182 296.987,78.9164 293.966,80.3447 290.854,82.1266 287.653,84.2839 284.366,86.8366 
 280.996,89.8027 277.546,93.1975 274.018,97.0339 270.416,101.322 266.744,106.067 263.003,111.273 259.198,116.938 255.331,123.059 251.406,129.626 247.426,136.627 
 243.395,144.044 239.317,151.856 235.194,160.039 231.03,168.563 226.83,177.395 222.595,186.5 218.331,195.838 214.041,205.367 209.728,215.042 205.397,224.817 
 201.05,234.644 196.692,244.474 192.327,254.258 187.958,263.945 183.589,273.487 179.223,282.837 174.866,291.948 170.519,300.776 166.187,309.279 161.875,317.42 
 157.584,325.163 153.32,332.476 149.086,339.332 144.885,345.707 140.722,351.584 136.599,356.946 132.52,361.783 128.49,366.091 124.51,369.867 120.585,373.115 
 116.718,375.842 112.913,378.057 109.172,379.776 105.499,381.015 101.898,381.796 98.3702,382.142 94.9199,382.078 91.5498,381.632 88.2628,380.833 85.0617,379.71 
 81.9493,378.295 78.9283,376.62 76.0013,374.717 73.1708,372.617 70.4392,370.353 67.809,367.955 65.2823,365.453 62.8613,362.879 60.5482,360.259 58.3449,357.62 
 56.2534,354.99 54.2753,352.392 52.4125,349.848 50.6665,347.381 49.0389,345.011 47.531,342.755 46.1442,340.63 44.8796,338.651 43.7384,336.833 42.7214,335.187 
 41.8297,333.724 41.0639,332.453 40.4247,331.382 39.9127,330.518 39.5283,329.865 39.2719,329.427 39.1436,329.208 
 "/>
<polyline clip-path="url(#clip2102)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 580.315,252.716 580.204,252.906 579.982,253.285 579.649,253.851 579.206,254.601 578.652,255.531 577.989,256.636 577.217,257.91 576.337,259.346 575.349,260.936 
 574.255,262.67 573.055,264.539 571.751,266.531 570.343,268.632 568.833,270.83 567.223,273.11 565.514,275.456 563.707,277.85 561.804,280.275 559.807,282.711 
 557.717,285.138 555.537,287.536 553.269,289.883 550.914,292.156 548.475,294.332 545.954,296.388 543.353,298.299 540.675,300.043 537.921,301.594 535.096,302.93 
 532.2,304.028 529.238,304.863 526.211,305.416 523.122,305.664 519.974,305.59 516.77,305.174 513.513,304.401 510.206,303.256 506.852,301.727 503.454,299.805 
 500.014,297.482 496.537,294.753 493.026,291.616 489.482,288.072 485.911,284.124 482.315,279.778 478.697,275.044 475.06,269.933 471.409,264.46 467.746,258.643 
 464.074,252.501 460.398,246.056 456.719,239.332 453.043,232.355 449.371,225.153 445.708,217.755 442.057,210.191 438.42,202.491 434.802,194.688 431.206,186.812 
 427.635,178.896 424.092,170.97 420.58,163.064 417.103,155.21 413.664,147.434 410.265,139.764 406.911,132.225 403.604,124.841 400.347,117.635 397.143,110.625 
 393.995,103.829 390.907,97.2637 387.879,90.9414 384.917,84.8733 382.021,79.0685 379.196,73.5336 376.443,68.2733 373.764,63.2903 371.163,58.5852 368.642,54.1571 
 366.203,50.0033 363.848,46.1195 361.58,42.5004 359.4,39.1391 357.31,36.0279 355.313,33.1583 353.41,30.5209 351.603,28.106 349.894,25.9032 348.284,23.9022 
 346.774,22.0926 345.367,20.4637 344.062,19.0055 342.862,17.7079 341.768,16.5615 340.78,15.5573 339.9,14.6871 339.128,13.943 338.465,13.3185 337.912,12.8073 
 337.468,12.4047 337.135,12.1063 336.913,11.9091 336.802,11.811 
 "/>
 
 
 
 
 y1

In [3]:
# whats the first deriv of that function at at 0.785?
println(f'(0.785))
# and close to the jump, at 0.100001?
println(f'(0.1000001))
# integral of f?
g1 = cumsum(f)
g = g1 + f(-1)
integral = norm(f-g)
integral

-0.006370611665946768
-1.5576748429320766


2.152565021925387


* The main purpose of this package is to manipulate analytic functions, i.e. function with an algebraic representation.
* There is the possibility to supply a set of data points and fit a polynomial:

In [4]:
S = Chebyshev(1..2);
n = 100; m = 50;
p = linspace(1,2,n);   # a non-default grid
v = exp.(p);           # values at the non-default grid
V = Array(Float64,n,m); # Create a Vandermonde matrix by evaluating the basis at the grid
for k = 1:m
    V[:,k] = Fun(S,[zeros(k-1);1]).(p)
end
f = Fun(S,V\v);
f(1.1)
3.004166023946434
exp(1.1)
3.0041660239464334

# Check out more examples on the [docs](http://juliaapproximation.github.io/ApproxFun.jl/stable/faq.html)

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] Array(::Type{Float64}, ::Int64, ::Int64) at ./deprecated.jl:57
 [3] include_string(::String, ::String) at ./loading.jl:515
 [4] include_string(::Module, ::String, ::String) at /Users/74097/.julia/v0.6/Compat/src/Compat.jl:174
 [5] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/74097/.julia/v0.6/IJulia/src/execute_request.jl:158
 [6] (::Compat.#inner#17{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /Users/74097/.julia/v0.6/Compat/src/Compat.jl:488
 [7] eventloop(::ZMQ.Socket) at /Users/74097/.julia/v0.6/IJulia/src/eventloop.jl:8
 [8] (::IJulia.##14#17)() at ./task.jl:335
while loading In[4], in expression starting on line 5


3.0041660239464334

## Splines: Piecewise Polynomial Approximation

* Splines are a finite element method, i.e. there are regions of the function domain where some basis functions are zero.
* As such, they provide a very flexible framework for approximation instead of high-order polynomials.
	* Keep in mind that Polynomials basis functions are non-zero on the entire domain. Remember the Vandermonde matrix.
* They bring some element of local approximation back into our framework. What happens at one end of the domain to the function is not important to what happens at the other end.
* Looking back at the previous plot of random data: we are searching for **one** polynomial to fit **all** those wiggles. A spline will allow us to design **different** polynomials in different parts of the domain.


### Splines: Basic Setup

* The fundamental building block is the *knot vector*, or the *breakpoints vector* $\mathbf{z}$ of length $p$. An element of $\mathbf{z}$ is $z_i$.
* $\mathbf{z}$ is ordered in ascending order.
* $\mathbf{z}$ spans the domain $[a,b]$ of our function, and we have that $a=z_1,b=z_p$
* A spline is of *order k* if the polynomial segments are k-th order polynomials.
* Literature: [@deboor] <cite data-cite=deboo></cite> is the definitive reference for splines.

### Splines: Characterization

* Given $p$ knots, there are $p-1$ polynomial segments of order $k$, each characterized by $k+1$ coefficients, i.e. a total of $(p-1)(k+1)$ parameters.
* However, we also require the spline to be continuous and differentiable of degree $k-1$ at the $p-2$ interior breakpoints.
* Imposing that uses up an additional $k(p-2)$ conditions.
* We are left with $n = (p-1)(k+1) - k(p-2) = p+k-1$ free parameters.
* A Spline of order $k$ and $p$ knots can thus be written as a linear combination of it's $n = p+k-1$ basis functions.


## B-Splines: Definition

* We mostly use Basis Splines, or **B-Splines**.
* Here is a recursive definition of a B-Spline (and what is used in `ApproXD`):
* Denote the $j$-th basis function of degree $k$ with knot vector $\mathbf{z}$  at $x$ as $B_j^{k,\mathbf{z}} (x)$
* Again, there are $n = k + p - 1$ $B$'s (where $p$` = length(z)`)
* We can define $B_j^{k,\mathbf{z}} (x)$ recursively like this:

	$$ 
    B_j^{k,\mathbf{z}} (x) = \frac{x-z_{j-k}}{z_j - z_{j-k}} B_{j-1}^{k-1,\mathbf{z}} (x)  + \frac{z_{j+1}-x}{z_{j+1} - z_{j+1-k}} B_{j}^{k-1,\mathbf{z}} (x), j=1,\dots,n
    $$

* The recursion starts with

$$ B_j^{0,\mathbf{z}} (x) = \begin{cases}
    1 & \text{if }z_j \leq x <	z_{j+1}\\
    0 & \text{otherwise.}
    \end{cases}
$$

* For this formulation to work, we need to extend the knot vector for $j<1,j>p$:
	
$$ z_j = \begin{cases}
    a & \text{if }j \leq 1\\
    b & \text{if }j \geq p
    \end{cases} 
$$
        
* And we need to set the endpoints
	$$ B_0^{k-1,\mathbf{z}} = B_n^{k-1,\mathbf{z}} =0 $$
* You may see that this gives rise to a triangular computation strategy, as pointed out [here](http://www.cs.mtu.edu/~shene/COURSES/cs3621/NOTES/spline/B-spline/bspline-basis.html).



## B-Splines: Derivatives and Integrals

* This is another very nice thing about B-Splines.
* The derivative wrt to it's argument $x$ is

	$$ \frac{d B_j^{k,\mathbf{z}} (x)}{dx} = \frac{k}{z_j - z_{j-k}} B_{j-1}^{k-1,\mathbf{z}} (x)  + \frac{k}{z_{j+1} - z_{j+1-k}} B_{j}^{k-1,\mathbf{z}} (x), j=1,\dots,n
    $$
    
* Similarly, the Integral is just the sum over the basis functions:
	$$ \int_a^x B_j^{k,\mathbf{z}} (y) dy = \sum_{i=j}^n \frac{z_i - z_{i-k}}{k} B_{i+1}^{k+1,\mathbf{z}} (x)  $$



In [5]:
using ApproXD   
# Pkg.clone("https://github.com/floswald/ApproXD.jl")
bs = BSpline(7,3,0,1) #7 knots, degree 3 in [0,1]
# how many basis functions? (go back 1 slide.)
# getNumCoefs(bs)
B = full(getBasis(collect(linspace(0,1.0,500)),bs))
plot(linspace(0,1.0,500),B,layout=(3,3),grid=false,ylim=(-0.1,1.1),legend=false,linewidth=2,linecolor=:black)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 1.00 
 
<polyline clip-path="url(#clip2302)" style="stroke:#000000; stroke-width:2; stroke-opacity:1; fill:none" points="
 38.8117,19.9223 39.0953,22.8132 39.3789,25.6346 39.6625,28.3873 39.946,31.0722 40.2296,33.69 40.5132,36.2418 40.7968,38.7283 41.0803,41.1503 41.3639,43.5088 
 41.6475,45.8044 41.931,48.0382 42.2146,50.211 42.4982,52.3235 42.7818,54.3766 43.0653,56.3713 43.3489,58.3082 43.6325,60.1884 43.9161,62.0125 44.1996,63.7816 
 44.4832,65.4963 44.7668,67.1576 45.0504,68.7663 45.3339,70.3233 45.6175,71.8293 45.9011,73.2853 46.1846,74.6921 46.4682,76.0505 46.7518,77.3614 47.0354,78.6256 
 47.3189,79.844 47.6025,81.0174 47.8861,82.1467 48.1697,83.2326 48.4532,84.2762 48.7368,85.2781 49.0204,86.2393 49.304,87.1606 49.5875,88.0428 49.8711,88.8868 
 50.1547,89.6935 50.4383,90.4636 50.7218,91.1981 51.0054,91.8977 51.289,92.5633 51.5725,93.1958 51.8561,93.796 52.1397,94.3648 52.4233,94.903 52.7068,95.4114 
 52.9904,95.8909 53.274,96.3424 53.5576,96.7666 53.8411,97.1645 54.1247,97.5368 54.4083,97.8845 54.6919,98.2083 54.9754,98.5091 55.259,98.7879 55.5426,99.0453 
 55.8262,99.2823 56.1097,99.4996 56.3933,99.6983 56.6769,99.879 56.9604,100.043 57.244,100.19 57.5276,100.322 57.8112,100.44 58.0947,100.544 58.3783,100.635 
 58.6619,100.714 58.9455,100.782 59.229,100.839 59.5126,100.887 59.7962,100.927 60.0798,100.959 60.3633,100.984 60.6469,101.002 60.9305,101.016 61.214,101.025 
 61.4976,101.031 61.7812,101.034 62.0648,101.035 62.3483,101.035 62.6319,101.035 62.9155,101.035 63.1991,101.035 63.4826,101.035 63.7662,101.035 64.0498,101.035 
 64.3334,101.035 64.6169,101.035 64.9005,101.035 65.1841,101.035 65.4677,101.035 65.7512,101.035 66.0348,101.035 66.3184,101.035 66.6019,101.035 66.8855,101.035 
 67.1691,101.035 67.4527,101.035 67.7362,101.035 68.0198,101.035 68.3034,101.035 68.587,101.035 68.8705,101.035 69.1541,101.035 69.4377,101.035 69.7213,101.035 
 70.0048,101.035 70.2884,101.035 70.572,101.035 70.8556,101.035 71.1391,101.035 71.4227,101.035 71.7063,101.035 71.9898,101.035 72.2734,101.035 72.557,101.035 
 72.8406,101.035 73.1241,101.035 73.4077,101.035 73.6913,101.035 73.9749,101.035 74.2584,101.035 74.542,101.035 74.8256,101.035 75.1092,101.035 75.3927,101.035 
 75.6763,101.035 75.9599,101.035 76.2435,101.035 76.527,101.035 76.8106,101.035 77.0942,101.035 77.3777,101.035 77.6613,101.035 77.9449,101.035 78.2285,101.035 
 78.512,101.035 78.7956,101.035 79.0792,101.035 79.3628,101.035 79.6463,101.035 79.9299,101.035 80.2135,101.035 80.4971,101.035 80.7806,101.035 81.0642,101.035 
 81.3478,101.035 81.6313,101.035 81.9149,101.035 82.1985,101.035 82.4821,101.035 82.7656,101.035 83.0492,101.035 83.3328,101.035 83.6164,101.035 83.8999,101.035 
 84.1835,101.035 84.4671,101.035 84.7507,101.035 85.0342,101.035 85.3178,101.035 85.6014,101.035 85.885,101.035 86.1685,101.035 86.4521,101.035 86.7357,101.035 
 87.0192,101.035 87.3028,101.035 87.5864,101.035 87.87,101.035 88.1535,101.035 88.4371,101.035 88.7207,101.035 89.0043,101.035 89.2878,101.035 89.5714,101.035 
 89.855,101.035 90.1386,101.035 90.4221,101.035 90.7057,101.035 90.9893,101.035 91.2729,101.035 91.5564,101.035 91.84,101.035 92.1236,101.035 92.4071,101.035 
 92.6907,101.035 92.9743,101.035 93.2579,101.035 93.5414,101.035 93.825,101.035 94.1086,101.035 94.3922,101.035 94.6757,101.035 94.9593,101.035 95.2429,101.035 
 95.5265,101.035 95.81,101.035 96.0936,101.035 96.3772,101.035 96.6607,101.035 96.9443,101.035 97.2279,101.035 97.5115,101.035 97.795,101.035 98.0786,101.035 
 98.3622,101.035 98.6458,101.035 98.9293,101.035 99.2129,101.035 99.4965,101.035 99.7801,101.035 100.064,101.035 100.347,101.035 100.631,101.035 100.914,101.035 
 101.198,101.035 101.482,101.035 101.765,101.035 102.049,101.035 102.332,101.035 102.616,101.035 102.899,101.035 103.183,101.035 103.467,101

In [6]:
# Notice that placing each of those panels on top of each other generates a sparse matrix!
println(getBasis(collect(linspace(0,1.0,5)),bs))
plot(linspace(0,1.0,500),B,grid=false,ylim=(-0.1,1.1),legend=false)


  [1, 1]  =  1.0
  [1, 2]  =  0.0
  [2, 2]  =  0.03125
  [1, 3]  =  0.0
  [2, 3]  =  0.46875
  [1, 4]  =  0.0
  [2, 4]  =  0.479167
  [3, 4]  =  0.166667
  [2, 5]  =  0.0208333
  [3, 5]  =  0.666667
  [4, 5]  =  0.0208333
  [3, 6]  =  0.166667
  [4, 6]  =  0.479167
  [3, 7]  =  0.0
  [4, 7]  =  0.46875
  [4, 8]  =  0.03125
  [5, 9]  =  1.0


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 1.00 
 
<polyline clip-path="url(#clip2502)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 38.8117,42.1446 39.8969,52.9555 40.9821,63.5064 42.0673,73.8006 43.1524,83.8411 44.2376,93.6311 45.3228,103.174 46.408,112.472 47.4931,121.53 48.5783,130.35 
 49.6635,138.935 50.7487,147.288 51.8339,155.414 52.919,163.314 54.0042,170.992 55.0894,178.451 56.1746,185.695 57.2597,192.726 58.3449,199.548 59.4301,206.163 
 60.5153,212.576 61.6004,218.788 62.6856,224.804 63.7708,230.627 64.856,236.259 65.9412,241.704 67.0263,246.965 68.1115,252.045 69.1967,256.947 70.2819,261.675 
 71.367,266.231 72.4522,270.619 73.5374,274.842 74.6226,278.903 75.7077,282.806 76.7929,286.553 77.8781,290.147 78.9633,293.593 80.0485,296.892 81.1336,300.048 
 82.2188,303.065 83.304,305.945 84.3892,308.691 85.4743,311.308 86.5595,313.797 87.6447,316.162 88.7299,318.407 89.815,320.534 90.9002,322.546 91.9854,324.448 
 93.0706,326.241 94.1558,327.929 95.2409,329.516 96.3261,331.004 97.4113,332.396 98.4965,333.696 99.5816,334.907 100.667,336.032 101.752,337.075 102.837,338.037 
 103.922,338.924 105.008,339.736 106.093,340.479 107.178,341.155 108.263,341.767 109.348,342.318 110.433,342.812 111.519,343.252 112.604,343.64 113.689,343.981 
 114.774,344.276 115.859,344.53 116.944,344.746 118.03,344.926 119.115,345.074 120.2,345.193 121.285,345.286 122.37,345.356 123.456,345.407 124.541,345.442 
 125.626,345.463 126.711,345.475 127.796,345.479 128.881,345.48 129.967,345.48 131.052,345.48 132.137,345.48 133.222,345.48 134.307,345.48 135.392,345.48 
 136.478,345.48 137.563,345.48 138.648,345.48 139.733,345.48 140.818,345.48 141.904,345.48 142.989,345.48 144.074,345.48 145.159,345.48 146.244,345.48 
 147.329,345.48 148.415,345.48 149.5,345.48 150.585,345.48 151.67,345.48 152.755,345.48 153.84,345.48 154.926,345.48 156.011,345.48 157.096,345.48 
 158.181,345.48 159.266,345.48 160.352,345.48 161.437,345.48 162.522,345.48 163.607,345.48 164.692,345.48 165.777,345.48 166.863,345.48 167.948,345.48 
 169.033,345.48 170.118,345.48 171.203,345.48 172.288,345.48 173.374,345.48 174.459,345.48 175.544,345.48 176.629,345.48 177.714,345.48 178.8,345.48 
 179.885,345.48 180.97,345.48 182.055,345.48 183.14,345.48 184.225,345.48 185.311,345.48 186.396,345.48 187.481,345.48 188.566,345.48 189.651,345.48 
 190.736,345.48 191.822,345.48 192.907,345.48 193.992,345.48 195.077,345.48 196.162,345.48 197.248,345.48 198.333,345.48 199.418,345.48 200.503,345.48 
 201.588,345.48 202.673,345.48 203.759,345.48 204.844,345.48 205.929,345.48 207.014,345.48 208.099,345.48 209.184,345.48 210.27,345.48 211.355,345.48 
 212.44,345.48 213.525,345.48 214.61,345.48 215.696,345.48 216.781,345.48 217.866,345.48 218.951,345.48 220.036,345.48 221.121,345.48 222.207,345.48 
 223.292,345.48 224.377,345.48 225.462,345.48 226.547,345.48 227.632,345.48 228.718,345.48 229.803,345.48 230.888,345.48 231.973,345.48 233.058,345.48 
 234.144,345.48 235.229,345.48 236.314,345.48 237.399,345.48 238.484,345.48 239.569,345.48 240.655,345.48 241.74,345.48 242.825,345.48 243.91,345.48 
 244.995,345.48 246.081,345.48 247.166,345.48 248.251,345.48 249.336,345.48 250.421,345.48 251.506,345.48 252.592,345.48 253.677,345.48 254.762,345.48 
 255.847,345.48 256.932,345.48 258.017,345.48 259.103,345.48 260.188,345.48 261.273,345.48 262.358,345.48 263.443,345.48 264.529,345.48 265.614,345.48 
 266.699,345.48 267.784,345.48 268.869,345.48 269.954,345.48 271.04,345.48 272.125,345.48 273.21,345.48 274.295,345.48 275.38,345.48 276.465,345.48 
 277.551,345.48 278.636,345.48 279.721,345.48 280.806,345.48 281.891,345.48 282.977,345.48 284.062,345.48 285.147,345.48 286.232,345.48 287.317,345.48 
 288.402,345.48 289.488,345.48 290.573,345.48 291.658,345.48 292.743,345.48 293.828,345.48 294.913,345.48 295.999,345.48 

## Linear B-Spline: A useful special case

* This is *connecting the dots with a straight line*
* This may incur some approximation error if the underlying function is very curved between the dots.
* However, it has some benefits: 
	* it is shape-preserving, 
	* it is fast,
	* it is easy to build.
* For a linear spline with evenly spaced breakpoints, this becomes almost trivial.
	* Let's define $h = \frac{b-a}{n-1}$ as the distance between breakpoints.
	* Our basis function becomes very simple, giving us a measure of how far $x$ is from the next knot:
    
$$ \phi_j (x) = \begin{cases}
    1 - \frac{|x-z_j|}{h} & \text{if } |x-z_j| \leq h \\
    0                    & \text{otherwise}
    \end{cases} 
$$

* Notice that each interior basis function (i.e. not 0 and not $n$) has witdth $2h$.

In [7]:
bs = BSpline(9,1,0,1) #9 knots, degree 1 in [0,1]
B = full(getBasis(collect(linspace(0,1.0,500)),bs))
plot(linspace(0,1.0,500),B,layout=(3,3),grid=false,ylim=(-0.1,1.1),legend=false,linewidth=2,linecolor=:black)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 1.00 
 
<polyline clip-path="url(#clip2702)" style="stroke:#000000; stroke-width:2; stroke-opacity:1; fill:none" points="
 38.8117,19.9223 39.0953,21.2227 39.3789,22.5232 39.6625,23.8236 39.946,25.124 40.2296,26.4244 40.5132,27.7248 40.7968,29.0252 41.0803,30.3256 41.3639,31.626 
 41.6475,32.9264 41.931,34.2269 42.2146,35.5273 42.4982,36.8277 42.7818,38.1281 43.0653,39.4285 43.3489,40.7289 43.6325,42.0293 43.9161,43.3297 44.1996,44.6301 
 44.4832,45.9306 44.7668,47.231 45.0504,48.5314 45.3339,49.8318 45.6175,51.1322 45.9011,52.4326 46.1846,53.733 46.4682,55.0334 46.7518,56.3338 47.0354,57.6343 
 47.3189,58.9347 47.6025,60.2351 47.8861,61.5355 48.1697,62.8359 48.4532,64.1363 48.7368,65.4367 49.0204,66.7371 49.304,68.0375 49.5875,69.338 49.8711,70.6384 
 50.1547,71.9388 50.4383,73.2392 50.7218,74.5396 51.0054,75.84 51.289,77.1404 51.5725,78.4408 51.8561,79.7412 52.1397,81.0417 52.4233,82.3421 52.7068,83.6425 
 52.9904,84.9429 53.274,86.2433 53.5576,87.5437 53.8411,88.8441 54.1247,90.1445 54.4083,91.4449 54.6919,92.7454 54.9754,94.0458 55.259,95.3462 55.5426,96.6466 
 55.8262,97.947 56.1097,99.2474 56.3933,100.548 56.6769,101.035 56.9604,101.035 57.244,101.035 57.5276,101.035 57.8112,101.035 58.0947,101.035 58.3783,101.035 
 58.6619,101.035 58.9455,101.035 59.229,101.035 59.5126,101.035 59.7962,101.035 60.0798,101.035 60.3633,101.035 60.6469,101.035 60.9305,101.035 61.214,101.035 
 61.4976,101.035 61.7812,101.035 62.0648,101.035 62.3483,101.035 62.6319,101.035 62.9155,101.035 63.1991,101.035 63.4826,101.035 63.7662,101.035 64.0498,101.035 
 64.3334,101.035 64.6169,101.035 64.9005,101.035 65.1841,101.035 65.4677,101.035 65.7512,101.035 66.0348,101.035 66.3184,101.035 66.6019,101.035 66.8855,101.035 
 67.1691,101.035 67.4527,101.035 67.7362,101.035 68.0198,101.035 68.3034,101.035 68.587,101.035 68.8705,101.035 69.1541,101.035 69.4377,101.035 69.7213,101.035 
 70.0048,101.035 70.2884,101.035 70.572,101.035 70.8556,101.035 71.1391,101.035 71.4227,101.035 71.7063,101.035 71.9898,101.035 72.2734,101.035 72.557,101.035 
 72.8406,101.035 73.1241,101.035 73.4077,101.035 73.6913,101.035 73.9749,101.035 74.2584,101.035 74.542,101.035 74.8256,101.035 75.1092,101.035 75.3927,101.035 
 75.6763,101.035 75.9599,101.035 76.2435,101.035 76.527,101.035 76.8106,101.035 77.0942,101.035 77.3777,101.035 77.6613,101.035 77.9449,101.035 78.2285,101.035 
 78.512,101.035 78.7956,101.035 79.0792,101.035 79.3628,101.035 79.6463,101.035 79.9299,101.035 80.2135,101.035 80.4971,101.035 80.7806,101.035 81.0642,101.035 
 81.3478,101.035 81.6313,101.035 81.9149,101.035 82.1985,101.035 82.4821,101.035 82.7656,101.035 83.0492,101.035 83.3328,101.035 83.6164,101.035 83.8999,101.035 
 84.1835,101.035 84.4671,101.035 84.7507,101.035 85.0342,101.035 85.3178,101.035 85.6014,101.035 85.885,101.035 86.1685,101.035 86.4521,101.035 86.7357,101.035 
 87.0192,101.035 87.3028,101.035 87.5864,101.035 87.87,101.035 88.1535,101.035 88.4371,101.035 88.7207,101.035 89.0043,101.035 89.2878,101.035 89.5714,101.035 
 89.855,101.035 90.1386,101.035 90.4221,101.035 90.7057,101.035 90.9893,101.035 91.2729,101.035 91.5564,101.035 91.84,101.035 92.1236,101.035 92.4071,101.035 
 92.6907,101.035 92.9743,101.035 93.2579,101.035 93.5414,101.035 93.825,101.035 94.1086,101.035 94.3922,101.035 94.6757,101.035 94.9593,101.035 95.2429,101.035 
 95.5265,101.035 95.81,101.035 96.0936,101.035 96.3772,101.035 96.6607,101.035 96.9443,101.035 97.2279,101.035 97.5115,101.035 97.795,101.035 98.0786,101.035 
 98.3622,101.035 98.6458,101.035 98.9293,101.035 99.2129,101.035 99.4965,101.035 99.7801,101.035 100.064,101.035 100.347,101.035 100.631,101.035 100.914,101.035 
 101.198,101.035 101.482,101.035 101.765,101.035 102.049,101.035 102.332,101.035 102.616,101.035 102.899,101.035 103.183,101.035 103.467,10

## Linear B-Spline: Evaluation

* In order to evaluate the linear interpolator, we need to know only one thing: Which knot span is active, i.e. what is $j$ s.t. $x\in [z_j, z_{j+1}]$?
* This is a classic problem in computer science. Binary search.
* `julia` implements [`searchsortedlast`](http://docs.julialang.org/en/release-0.4/stdlib/sort/#Base.searchsortedlast).	
* Once we know $j$, it's easy to get the interpolated value as
	$$ \hat{f}(x) = \frac{x-z_j}{h}  f(z_{j+1}) + \frac{z_{j+1}-x}{h}  f(z_{j}) $$




## The Importance of Knot Placement

* We just talked about *equally spaced knots*. This is just a special case.
* B-Splines give us the flexibility to place the knots where we want.
* Contrary to Polynomial interpolations (where we cannot choose the evaluation nodes), this is very helpful in cases where we know that a function is very curved in a particular region.
* Canonical Example: Runge's function: $f(x) = (1+25x^2)^{-1}$.
* Also: If you know that your function has a kink (i.e. a discontinuous first derivative) at $\hat{x}$, then you can stack breakpoints on top of each other *at* $\hat{x}$

In [8]:
h(x) = (1+25x^2)^(-1)
plot(h)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
<polyline clip-path="url(#clip2902)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 32.2221,375.813 82.8523,375.51 113.279,375.202 139.004,374.803 166.949,374.098 192.712,372.946 206.522,371.939 220.332,370.424 236.333,367.48 252.333,361.734 
 259.443,357.271 266.553,350.485 270.107,345.701 273.662,339.516 277.217,331.361 280.772,320.364 282.331,314.324 283.889,307.334 285.448,299.208 287.006,289.719 
 288.565,278.593 290.124,265.511 291.682,250.101 293.241,231.963 294.8,210.702 296.358,186.02 297.917,157.876 299.476,126.749 301.034,93.9881 302.593,62.1493 
 303.372,47.7239 304.151,35.0238 304.931,24.6101 305.71,16.9937 306.56,12.3474 307.409,11.811 308.259,15.4208 309.109,22.9358 309.958,33.8766 310.808,47.5982 
 311.658,63.3771 312.507,80.4924 314.207,116.214 315.906,150.833 317.605,182.189 319.305,209.476 321.004,232.688 322.703,252.206 324.403,268.541 326.102,282.21 
 327.801,293.676 329.501,303.332 331.2,311.505 332.899,318.459 336.318,329.589 339.737,337.903 343.156,344.247 346.575,349.18 353.412,356.216 360.25,360.872 
 375.37,366.835 390.49,369.939 404.876,371.683 419.263,372.809 449.179,374.161 474.363,374.787 505.109,375.253 534.011,375.531 580.315,375.806 
 "/>
 
 
 
 
 y1

## Interpolation with [`Interpolations.jl`](https://github.com/JuliaMath/Interpolations.jl)


```julia
# Nearest-neighbor interpolation
itp = interpolate(a, BSpline(Constant()), OnCell())
v = itp[5.4]   # returns a[5]

# (Multi)linear interpolation
itp = interpolate(A, BSpline(Linear()), OnGrid())
v = itp[3.2, 4.1]  # returns 0.9*(0.8*A[3,4]+0.2*A[4,4]) + 0.1*(0.8*A[3,5]+0.2*A[4,5])

# Quadratic interpolation with reflecting boundary conditions
# Quadratic is the lowest order that has continuous gradient
itp = interpolate(A, BSpline(Quadratic(Reflect())), OnCell())

# Linear interpolation in the first dimension, and no interpolation (just lookup) in the second
itp = interpolate(A, (BSpline(Linear()), NoInterp()), OnGrid())
v = itp[3.65, 5]  # returns  0.35*A[3,5] + 0.65*A[4,5]
```

In [9]:
# In-place interpolation
# restart kernel!
using Interpolations
A = rand(10,5)
itp = interpolate!(A, Interpolations.BSpline(Quadratic(InPlace())), OnCell())
itp[0.2,0.9]

0.7067061048116161

* Very often we need a **gridded** interpolation. I.e. we supply the function values on an irregular grid.
* For this occasion, the `GriddedInterpolation` type is useful.
* For now this only works in 3 modes: 
    * Gridded(Linear())
    * Gridded(Constant())  nearest neighbor
    * NoInterp (you must supply index ON grid)

In [10]:
A = rand(20)
A_x = collect(1.0:2.0:40.0)
knots = (A_x,)
itp = interpolate(knots, A, Gridded(Linear()))
itp[2.0] 

0.8488726676613095

In [11]:
# 2D
A = rand(8,20)
knots = ([x^2 for x = 1:8], [0.2y for y = 1:20])
itp = interpolate(knots, A, Gridded(Linear()))
itp[4,4.1]  

0.8378654767599323

In [12]:
# we can mix modes across dimensions!
itp = interpolate(knots, A, (Gridded(Linear()),Gridded(Constant())))
itp[4,4.1]

0.6823361999844295

* What about vector valued interpolations?
* Suppose we have a function $f : \mathbb{R} \mapsto \mathbb{R}^2$
* Economics example:

$$  
f(x) = \left(\begin{array}{c} \text{savings}(x) \\ \text{consumption}(x) \end{array} \right)
$$

In [13]:
using StaticArrays
# a = rand(2,200)  # like f(x) above
a = Float64[log(1+i)/j for i in 1:2, j in linspace(1,3,200)]
b = reinterpret(SVector{2,Float64}, a, (200,))
bitp = scale(interpolate(b, Interpolations.BSpline(Quadratic(Reflect())), OnCell()),linspace(1,3,200))

# just compute the sum to demonstrate speed
function foo(itp, X)
    s = zero(eltype(itp))
    for x in X
        s += itp[x]
    end
    s
end

X = rand(10^7)

# After warming up with one call to `foo(bitp, X)`:
foo(bitp, X);
@time foo(bitp, X)


  0.167800 seconds (85 allocations: 6.467 KiB)


2-element SVector{2,Float64}:
 -1.24712e8
 -1.97664e8

In [14]:
using Plots
v=bitp[1.5]
println("cons=$(v[1])")
println("save=$(v[2])")
plot(linspace(1,3,200),a',labels=["cons","save"],yticks=convert(Array{Float64},v))
vline!([1.5])
hline!([v[1],v[2]])


# say you have assets of 0.5:


cons=0.46209812681126966
save=0.7324082026493517


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 2.5 
 
 
 3.0 
 
 
 0.46209813 
 
 
 0.73240820 
 
<polyline clip-path="url(#clip3102)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 78.3495,181.932 80.872,184.825 83.3944,187.662 85.9168,190.443 88.4393,193.171 90.9617,195.846 93.4841,198.471 96.0066,201.047 98.529,203.574 101.051,206.055 
 103.574,208.491 106.096,210.882 108.619,213.231 111.141,215.538 113.664,217.804 116.186,220.031 118.709,222.219 121.231,224.369 123.753,226.483 126.276,228.561 
 128.798,230.605 131.321,232.614 133.843,234.591 136.366,236.535 138.888,238.448 141.41,240.33 143.933,242.182 146.455,244.004 148.978,245.799 151.5,247.565 
 154.023,249.304 156.545,251.016 159.068,252.702 161.59,254.363 164.112,255.999 166.635,257.61 169.157,259.198 171.68,260.763 174.202,262.304 176.725,263.824 
 179.247,265.322 181.77,266.798 184.292,268.254 186.814,269.689 189.337,271.104 191.859,272.499 194.382,273.876 196.904,275.233 199.427,276.572 201.949,277.893 
 204.471,279.197 206.994,280.483 209.516,281.752 212.039,283.004 214.561,284.241 217.084,285.461 219.606,286.665 222.129,287.854 224.651,289.028 227.173,290.187 
 229.696,291.332 232.218,292.462 234.741,293.579 237.263,294.681 239.786,295.77 242.308,296.846 244.831,297.909 247.353,298.959 249.875,299.997 252.398,301.022 
 254.92,302.035 257.443,303.037 259.965,304.026 262.488,305.004 265.01,305.971 267.532,306.927 270.055,307.872 272.577,308.806 275.1,309.73 277.622,310.643 
 280.145,311.546 282.667,312.439 285.19,313.323 287.712,314.196 290.234,315.06 292.757,315.915 295.279,316.76 297.802,317.597 300.324,318.424 302.847,319.243 
 305.369,320.053 307.892,320.855 310.414,321.648 312.936,322.433 315.459,323.21 317.981,323.979 320.504,324.74 323.026,325.493 325.549,326.239 328.071,326.977 
 330.593,327.707 333.116,328.431 335.638,329.147 338.161,329.856 340.683,330.558 343.206,331.254 345.728,331.942 348.251,332.624 350.773,333.3 353.295,333.968 
 355.818,334.631 358.34,335.287 360.863,335.937 363.385,336.581 365.908,337.219 368.43,337.851 370.952,338.477 373.475,339.097 375.997,339.711 378.52,340.32 
 381.042,340.924 383.565,341.521 386.087,342.114 388.61,342.701 391.132,343.283 393.654,343.86 396.177,344.431 398.699,344.998 401.222,345.559 403.744,346.116 
 406.267,346.668 408.789,347.215 411.312,347.757 413.834,348.295 416.356,348.828 418.879,349.356 421.401,349.88 423.924,350.4 426.446,350.915 428.969,351.426 
 431.491,351.932 434.013,352.435 436.536,352.933 439.058,353.427 441.581,353.917 444.103,354.403 446.626,354.885 449.148,355.363 451.671,355.838 454.193,356.308 
 456.715,356.775 459.238,357.238 461.76,357.697 464.283,358.153 466.805,358.605 469.328,359.054 471.85,359.499 474.373,359.94 476.895,360.378 479.417,360.813 
 481.94,361.244 484.462,361.672 486.985,362.097 489.507,362.519 492.03,362.937 494.552,363.352 497.074,363.764 499.597,364.173 502.119,364.579 504.642,364.982 
 507.164,365.382 509.687,365.779 512.209,366.173 514.732,366.564 517.254,366.952 519.776,367.338 522.299,367.72 524.821,368.1 527.344,368.477 529.866,368.852 
 532.389,369.224 534.911,369.593 537.433,369.959 539.956,370.323 542.478,370.684 545.001,371.043 547.523,371.399 550.046,371.753 552.568,372.104 555.091,372.453 
 557.613,372.8 560.135,373.144 562.658,373.486 565.18,373.825 567.703,374.162 570.225,374.497 572.748,374.829 575.27,375.16 577.793,375.488 580.315,375.813 
 
 "/>
<polyline clip-path="url(#clip3102)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 78.3495,11.811 80.872,16.3975 83.3944,20.8937 85.9168,25.302 88.4393,29.6253 90.9617,33.8657 93.4841,38.0258 96.0066,42.1078 98.529,46.1138 101.051,50.046 
 103.574,53.9063 106.096,57.6968 108.619,61.4193 111.141,65.0756 113.664,68.6675 116.186,72.1966 118.709,75.6647 121.231,79.0731 123.753,82.4236 126.276,85.7176 
 128.798,88.9564 131.321,92.1414 133.843,95.274 136.366,98.3


## The CompEcon Toolbox of Miranda and Fackler

* another good alternative:
* [CompEcon.jl](https://github.com/spencerlyon2/CompEcon.jl)



## Mulitidimensional Approximation

* Up to now, most of what we did was in one dimesion.
* Economic problems *often* have more dimension than that.
	* The number of state variables in your value functions are the number of dimensions.
* We can readily extend what we learned into more dimensions.
* However, we will quickly run into feasibility problems: hello *curse of dimensionality*.



## Tensor Product of univariate Basis Functions: Product Rule

* One possibility is to approximate e.g. the 2D function $f(x,y)$ by
	$$ \hat{f}(x,y) = \sum_{i=1}^n \sum_{j=1}^m c_{i,j} \phi_i^x(x) \phi_j^y(y)  $$
	* here $\phi_i^x$ is the basis function in $x$ space, 
	* you can see that the coefficient vector $c_{i,j}$ is indexed in two dimensions now.
	* Notice that our initial notation was general enough to encompass this case, as we defined the basis functions as $\mathbb{R}^d \mapsto \mathbb{R}$. So with the product rule, this mapping is just given by $\phi_i^x(x) \phi_j^y(y)$.
* This formulation requires that we take the product of $\phi_i^x(x), \phi_j^y(y)$ at *all* combinations of their indices, as is clear from the summations.
* This is equivalent to the tensor product between $\phi_i^x$ and $\phi_j^y$.

## Computing Coefficients from Tensor Product Spaces 

* Extending this into $D$ dimensions, where in each dim $i$ we have $n_i$ basis functions, we get
	$$ \hat{f}(x_1,x_2,\dots,x_D) = \sum_{i_1=1}^{n_1} \sum_{i_2=1}^{n_2} \dots  \sum_{i_D=1}^{n_D} c_{i_1,i_2,\dots,i_D} \phi_{i_1}(x_1) \phi_{i_2}(x_2) \dots \phi_{i_D}(x_D)  $$ 
* In Vector notation
	$$ 
	\hat{f}(x_1,x_2,\dots,x_D) =  \left[ \phi_{D}(x_D) \otimes \phi_{D-1}(x_{D-1})  \otimes \dots  \otimes  \phi_{1}(x_1) \right]  c $$
	where $c$ is is an $n=\Pi_{i=1}^D n_i$ column vector
* The solution is the interpolation equation as before,
	$$ \begin{aligned}\Phi c =& y \\
					\Phi   =& \Phi_D \otimes \Phi_{D-1} \otimes \dots \otimes \Phi_{1} \end{aligned} $$



### The Problem with Tensor Product of univariate Basis Functions

* What's the problem?
* Well, solving $\Phi c = y$ is hard. 
* If we have as many evaluation points as basis functions in each dimension, i.e. if each single $\Phi_i$ is a square matrix, $\Phi$ is of size (n,n). 
* Inverting this is *extremely* hard even for moderately sized problems.
* Sometimes it's not even possible to allocate $\Phi$ in memory.
* Here it's important to remember the sparsity structure of a spline basis function.

In [15]:
using ApproXD
bs = ApproXD.BSpline(7,3,0,1) #7 knots, degree 3 in [0,1]
n = 500
eval_points = collect(linspace(0,1.0,n))
B = full(ApproXD.getBasis(eval_points,bs))
ys = [string("Basis",i) for i = 1:8]
xs = eval_points
heatmap(xs,ys,B')

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 1.00 
 
 
 Basis1 
 
 
 Basis2 
 
 
 Basis3 
 
 
 Basis4 
 
 
 Basis5 
 
 
 Basis6 
 
 
 Basis7 
 
 
 Basis8 
 
 
<image width="469" height="364" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAdUAAAFsCAYAAACafCNKAAAPAklEQVR4nO3XXY8cyXmm4eeNLPYM
BWsN7/r/H/ln+djAAusVxiPNkN2ZET7IrOpqDi1o4dfQwrouIjo+myJYHNzqSm4rADzUn9ml6jvn
dX19vqvruK7zSjKSet6fo6pSGUmN6/VI1XjMueatbtfZlsrIyJaq+7xly5aRLSO3jGzZ7vPa8rJe
sl2/bterx8uqbFW5XeNc5zwfyY9bcqvkNpJPldzGuvYrn2pd+/W0n7nVyjZmfhjHub/GVtf87aiZ
ca2r1rnejoxaqW/G+Ve7fvPRZCVrVVYq8xhZqzLXyJyVOUfmHDnmlv0Y2eeW/djyNre8Hed4PbZ8
nVt+fvuUL8eWL8fIr8fIr0fllz35ZU9+PVb+uM/8cb7l5/yafxs/5ef8a37Z/29+ffs/Gf/Jf3sA
wEVUAaCJqAJAE1EFgCaiCgBNRBUAmogqADQRVQBoIqoA0ERUAaCJqAJAE1EFgCaiCgBNRBUAmogq
ADQRVQBoIqoA0ERUAaCJqAJAE1EFgCaiCgBNRBUAmogqADQRVQBoIqoA0ERUAaCJqAJAE1EFgCai
CgBNRBUAmogqADQRVQBoIqoA0ERUAaCJqAJAE1EFgCaiCgBNRBUAmogqADQRVQBoIqoA0ERUAaCJ
qAJAE1EFgCaiCgBNKrmtv/YfAvivVX9mdx7Vd+7r6eSbs6rHuh7/37we5+f791F1Pxupep9HbklV
KiNV45wf60ple5yPx/2WUSO3vKSyZcuWcR/r3I1s2dY1f3hRGRl5yZat6mnkN/OoXPvkVsk2zvXL
WO9ntbJd+61WbrXOs+cxznnUysuYGbWy1XzcjWs9aj7242lfNTPG+f23caRqZVznVesaSWV92Ofa
5+nue1Yqua7WOj+vtc71fcw5Mr9ZzzUy57k/1rjmyjFH5ho55jm+zi3HHNmvu2NVjlXZr7N9Vfb5
zbwq+0y+HJV9JfvMh/lYyT5X9vU05sqemT1H9sx8ydfstWfPfs1vOZ7H2jPvI3vmOjLXnrX2HOs1
cx1Z60gy3+esZM2szJx/aStZ53z/2/WTKgA0EVUAaCKqANBEVAGgiagCQBNRBYAmogoATUQVAJqI
KgA0EVUAaCKqANBEVAGgiagCQBNRBYAmogoATUQVAJqIKgA0EVUAaCKqANBEVAGgiagCQBNRBYAm
ogoATUQVAJqIKgA0EVUAaCKqANBEVAGgiagCQBNRBYAmogoATUQVAJqIKgA0EVUAaCKqANBEVAGg
iagCQBNRBYAmogoATUQVAJqIKgA0EVUAaCKqANBEVAGgSSW39df+Q8Bfqv7M7jyq79zVb76ec13b
+009ndd18nSWSlWlMj6ua1wvR6q2VI1rfc1P6/FYn+9Gtut0yy0v1/6WsUa2bOft2t7X14tz1DlX
ZavKS41slWufjGt+jHHOt+ezWrmN5GWs3Gplq5Vt5H1dz+czW62Mx93MNlZexnHdzcebbcyM62yM
89241mPMjFqpMXPbjoxxrkfN1FgZY6au+3N+Wt/3T/fnx7WSrMfHe+6/Y1XWfV73uR7zYz3f92uO
rFmZ1/o4tnM/R+YcWetaX/NaI3NWjjky1zjna/923HKskWPWOT+vH2fn/9ZxjX2+r7/O8/6YlX3l
Ok/2az5mrv05P5+9zXXus3Ks+0iONc+zzGuszDVz1HHu68hb3jJrZmZ/vHof85zXzMqRtWZWzjHX
9TteZ3m6W2s9nZ2/7utc98lMrpPrA3uM8xO+9uu+fp/X420+nL8vv/dvZH378v+Zn1QBoImoAkAT
UQWAJqIKAE1EFQCaiCoANBFVAGgiqgDQRFQBoImoAkATUQWAJqIKAE1EFQCaiCoANBFVAGgiqgDQ
RFQBoImoAkATUQWAJqIKAE1EFQCaiCoANBFVAGgiqgDQRFQBoImoAkATUQWAJqIKAE1EFQCaiCoA
NBFVAGgiqgDQRFQBoImoAkATUQWAJqIKAE1EFQCaiCoANBFVAGgiqgDQRFQBoImoAkATUQWAJqIK
AE1EFQCaVHJbf+0/xN+a+gtOUvWd+7q+1m/P676vx6vHqKQy3u+rkozzZVXqsT7nykjVON9UZdTt
enOej4xUtlSdZ6PO/fjO+LReMjKyZcu2Hqvr9vyux3dXvY9UXkZlq1xnyajkNnKdvY9bJdtY1/qc
f9jmdb+us5mt1vXumzHm0/3My3Zkq5kx3s/G9WZc61HrWq+Mmqlr3m5HxpipOu/q6b4e80qu+T5S
6/0u6+kj/Q/+81yVlfNpUlmrsmad508j8/1urZE1K/M+X2dzjhz79n6+RuaszDky1zXPkeP+do4c
1/mxRl6P7XF2rPG0fhqzHvf74zz5eowcK9dZcszKvnKun8Y+k/nYrxwreZ0rR9a1v0ZmZs75eMzX
17qvZ97qNfM6fx4rR+aaWZlZ6/473dfnmGvPWiv5cL6Sdc4rM1nn6nxzfaaPuzz2uV499u8f6mNe
3+zf5yTre/8+1p/Z8V/NT6oA0ERUAaCJqAJAE1EFgCaiCgBNRBUAmogqADQRVQBoIqoA0ERUAaCJ
qAJAE1EFgCaiCgBNRBUAmogqADQRVQBoIqoA0ERUAaCJqAJAE1EFgCaiCgBNRBUAmogqADQRVQBo
IqoA0ERUAaCJqAJAE1EFgCaiCgBNRBUAmogqADQRVQBoIqoA0ERUAaCJqAJAE1EFgCaiCgBNRBUA
mogqADQRVQBoIqoA0ERUAaCJqAJAE1EFgCaV3NZ/8jf4zerDbX2zf1rXY1/fvK/r/n193j2fjVRd
cyqpcf2O9/V5XznX72cjI1tGbY+7qpHKlnG/y31+HreMNfIpn7Kt7Txd5/ds53dfo7LVyFbJVnWO
VLZKPo1z3kau++R2zfezW63rbp13Y2WrlR/GzFbn+nadbbUy6r6e19un+Wn9aduzjZkx1jnXfOyr
ZsaYGbUyxkxd8xgzNVa27Uhd9zVW6npT1z73db2vzznnnHP+nrWuz36dn+talazzfM1xzef542zW
h/t5nc01subInJX92M71dT/nyFwjxzz3xzrPjjlyrLrmc/86txyzzv2qzFVPbyr7fX9fzzzWr7Oy
X/tjJcdK9ms+372Puda1PufXNa/1ypGZI/d5Zl7zkSOzjqf1zMyemSN7Xj+8XmtmXfPMzMq8zr5Z
r5m1jut8JY/zlWR9WN/Huq/Xt2fXh7ny9D7XffL0YX9cr+/9+/j+d8D/z/ykCgBNRBUAmogqADQR
VQBoIqoA0ERUAaCJqAJAE1EFgCaiCgBNRBUAmogqADQRVQBoIqoA0ERUAaCJqAJAE1EFgCaiCgBN
RBUAmogqADQRVQBoIqoA0ERUAaCJqAJAE1EFgCaiCgBNRBUAmogqADQRVQBoIqoA0ERUAaCJqAJA
E1EFgCaiCgBNRBUAmogqADQRVQBoIqoA0ERUAaCJqAJAE1EFgCaiCgBNRBUAmogqADQRVQBoIqoA
0KSS23ranF+rrl2lMpIaqVSSkartMY/astVLqm4ZdcuoLSP39S1b3bLl02Pc8im3dcstt9zWLT/m
h9wycst2zqNyq6cxKlslt0pu4+P847a

* This is a cubic spline basis. at most $k+1=4$ basis are non-zero for any $x$.

In [16]:
heatmap(xs,ys,B'.>0)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 1.00 
 
 
 Basis1 
 
 
 Basis2 
 
 
 Basis3 
 
 
 Basis4 
 
 
 Basis5 
 
 
 Basis6 
 
 
 Basis7 
 
 
 Basis8 
 
 
<image width="469" height="364" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAdUAAAFsCAYAAACafCNKAAAF+ElEQVR4nO3XsW3DUBQEQdL4hbI0
Bi6TAF2CFKzwLGmmgssWt2/bujeAD3Xd5/QEvsjP9AAA+BSiCgARUQWAiKgCQERUASAiqgAQEVUA
iIgqAEREFQAiogoAEVEFgIioAkBEVAEgIqoAEBFVAIiIKgBERBUAIqIKABFRBYCIqAJARFQBICKq
ABARVQCIiCoAREQVACKiCgARUQWAiKgCQERUASAiqgAQEVUAiIgqAEREFQAiogoAEVEFgIioAkBE
VAEgIqoAEBFVAIiIKgBERBUAIqIKABFRBYCIqAJARFQBILKmBwDPue5zegLwgKcKABFRBYCIqAJA
RFQBICKqABARVQCIiCoAREQVACKiCgARUQWAiKgCQERUASAiqgAQEVUAiIgqAEREFQAiogoAEVEF
gIioAkBEVAEgIqoAEBFVAIiIKgBERBUAIqIKABFRBYCIqAJARFQBICKqABARVQCIiCoAREQVACKi
CgARUQWAiKgCQERUASAiqgAQEVUAiIgqAEREFQAiogoAEVEFgIioAkBEVAEgsqYH8H2u+5yeAPAS
nioAREQVACKiCgARUQWAiKgCQERUASAiqgAQEVUAiIgqAEREFQAiogoAEVEFgIioAkBEVAEgIqoA
EBFVAIiIKgBERBUAIqIKABFRBYCIqAJARFQBICKqABARVQCIiCoAREQVACKiCgARUQWAiKgCQERU
ASAiqgAQEVUAiIgqAEREFQAiogoAEVEFgIioAkBEVAEgIqoAEBFVAIiIKgBERBUAIqIKABFRBYCI
qAJAZE0PeGfXfU5PAHiJtR/TE96SpwoAEVEFgIioAkBEVAEgIqoAEBFVAIiIKgBERBUAIqIKABFR
BYCIqAJARFQBICKqABARVQCIiCoAREQVACKiCgARUQWAiKgCQERUASAiqgAQEVUAiIgqAEREFQAi
ogoAEVEFgIioAkBEVAEgIqoAEBFVAIiIKgBERBUAIqIKABFRBYCIqAJARFQBICKqABARVQCIiCoA
REQVACKiCgARUQWAiKgCQERUASCytm3brvscngHwGms/pifwRTxVAIiIKgBERBUAIqIKABFRBYCI
qAJARFQBICKqABARVQCIiCoAREQVACKiCgARUQWAiKgCQERUASAiqgAQEVUAiIgqAEREFQAiogoA
EVEFgIioAkBEVAEgIqoAEBFVAIiIKgBERBUAIqIKABFRBYCIqAJARFQBICKqABARVQCIiCoAREQV
ACKiCgARUQWAiKgCQERUASAiqgAQEVUAiIgqAEREFQAiogoAEVEFgMiaHgA8Z+3H9ATgAU8VACKi
CgARUQWAiKgCQERUASAiqgAQEVUAiIgqAEREFQAiogoAEVEFgIioAkBEVAEgIqoAEBFVAIiIKgBE
RBUAIqIKABFRBYCIqAJARFQBICKqABARVQCIiCoAREQVACKiCgARUQWAiKgCQERUASAiqgAQEVUA
iIgqAEREFQAiogoAEVEFgIioAkBEVAEgIqoAEBFVAIiIKgBERBUAIqIKABFRBYCIqAJAZE0P4Pus
/ZieAPASnioAREQVACKiCgARUQWAiKgCQERUASAiqgAQEVUAiIgqAEREFQAiogoAEVEFgIioAkBE
VAEgIqoAEBFVAIiIKgBERBUAIqIKABFRBYCIqAJARFQBICKqABARVQCIiCoAREQVACKiCgARUQWA
iKgCQERUASAiqgAQEVUAiIgqAEREFQAiogoAEVEFgIioAkBEVAEgIqoAEBFVAIiIKgBERBUAIqIK
ABFRBYCIqAJAZL/u33t6xLta+zE9AYB/xFMFgIioAkBEVAEgIqoAEBFVAIiIKgBERBUAIqIKABFR
BYCIqAJARFQBICKqABARVQCIiCoAREQVACKiCgARUQWAiKgCQERUASAiqgAQEVUAiIgqAEREFQAi
ogoAEVEFgIioAkBEVAEgIqoAEBFVAIiIKgBERBUAIqIKABFRBYCIqAJARFQBICKqABARVQCIiCoA
REQVACKiCgARUQWAiKgCQERUASAiqgAQ+QOQcxaEfv9e1AAAAABJRU5ErkJggg==
" transform="translate(51, 12)"/>
 
 
 
 
 
 
 
<image width="18" height="364" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAABIAAAFsCAYAAADBkelSAAACHUlEQVR4nO2c223EMAwEqUD9V5Eu
4yOvBPpjEAwIqoCF9kU5vljnyd8KYP0QIBERN+thgAoC4qhVzqVmcw0Te7BGHLWgkk0BgdQo1yZr
lH8Q0Ca7Wxy1Ux8GKBICOhAQaT+lESQ2aD9Gbe1vgU4mAiSkRgEpk824NjnZRvsh1yj7la6N1Qi0
35ZsnWtC+yORF1pxo3zUMNcYjZTUVqMeCMFBqUFAUGeN1HRA5Ds2CKjyIEBK+xlqt8qnEUSNAiLf
+Y+lFslgTdaImkc3itmUkJpxZm+ym3XLppHwOKI0Eh5HXPt1gw2035Zs0P6x7QeT7aO2rrVAtooY
2++z33aukf8PKSutUKNN9j8C5WrUraXWL18gZ2s09mlk8DzaZPdAQvtDRk1oP/TjupMaZT+Uo8Gl
NdpPAQmpYa7pks2VFhtsOo249iMwS+3NMs5sBEbZ/qXWA/ke2AfPbIya8WEUAtJVZO3vF2g/g6Oc
2XM1AqnZkg0eRxAQtSOlRmt/s4TUVqMXQAyOcmbr5pFu1N6CPqoRJntwaXU5MrZfl+zB1MCfM6Dv
sn3n2s7sF0A6apTYGDWhRkmVVpjswa5R1Ar6etmo0cGo2a4JoXYE2n909nPJttkPltZ2/xGYbOge
LZ/9Oo2WWr/A0kI34GA7Iv/KnksNm0eDNeKo6dq/rvVAVEWoHQmHf+kumcaoBfQmArwngnON0sjX
fs7+udQgoC/rNOfnsknqIwAAAABJRU5ErkJggg==
" transform="translate(532, 12)"/>
 
 
 0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 0.4 
 
 
 0.5 
 
 
 0.6 
 
 
 0.7 
 
 
 0.8 
 
 
 0.9 
 
 
 1.0 
 
<polyline clip-path="url(#clip3500)" style="stroke:#000000; stroke-width:1; stroke-opacity:1; fill:none" points="
 550.315,375.813 550.315,375.813 556.315,375.813 550.315,375.813 550.315,339.413 556.315,339.413 550.315,339.413 550.315,303.013 556.315,303.013 550.315,303.013 
 550.315,266.613 556.315,266.613 550.315,266.613 550.315,230.212 556.315,230.212 550.315,230.212 550.315,193.812 556.315,193.812 550.315,193.812 550.315,157.412 
 556.31

## Using Sparsity of Splines

* It may be better to store the splines in sparse format.
* Look at object `B` by typing `B` and `typeof(B)`
* There are sparse system solvers available.
* Creating and storing the inverse of $\Phi$ destroys the sparsity structure (inverse of a sparse matrix is not sparse), and may not be a good idea.
* Look back at [Computing coefficients form the tensor product](#compute-c)
* We only have to sum over the non-zero entries! Every other operation is pure cost.
* This is implemented in `ApproXD.jl` for example via

```julia
    function evalTensor2{T}(mat1::SparseMatrixCSC{T,Int64},
                            mat2::SparseMatrixCSC{T,Int64},
                            c::Vector{T})
```


## High Dimensional Functions: Introducing the Smolyak Grid

* This is a modification of the Tensor product rule. 
* It elemininates points from the full tensor product according to their *importance* for the quality of approximation.
* The user controls this quality parameter, thereby increasing/decreasing the size of the grid.
* [@jmmv] <cite data-cite=jmmv></cite>is a complete technical reference for this method.
* [@maliar-maliar] <cite data-cite=maliarmaliar></cite>chapter 4 is very good overview of this topic, and the basis of this part of the lecture.


## The Smolyak Grid in 2 Dimensions

* Approximation level $\mu \in \mathbb{N}$ governs the quality of the approximation.
* Start with a unidimensional grid of points $x$:
	$$ x = \left\{-1,\frac{-1}{\sqrt{2}},0,\frac{1}{\sqrt{2}},1\right\} $$
	which are 5 Chebyshev nodes (it's not important that those are Chebyshev nodes, any grid will work).
* A 2D tensor product $x\otimes x$ gives 25 grid points
	$$ x\otimes x=\left\{(-1,-1),(-1,\frac{-1}{\sqrt{2}}),\dots,(1,1)\right\} $$
* The Smolyak method proceeds differently.
* We construct three nested sets:

$$ \begin{array}{l}
		i=1 : S_1 = \{0\} \\
		i=2 : S_2 = \{0,-1,1\} \\
		i=3 : S_3 = \left\{-1,\frac{-1}{\sqrt{2}},0,\frac{1}{\sqrt{2}},1\right\}  \end{array} $$
        
* Then, we construct all possible 2D tensor products using elements from these nested sets in a table (next slide).
* Finally, we select only those elements of the table, that satisfy the Smolyak rule:
	$$ i_1 + i_2 \leq d + \mu $$
	where $i_1,i_2$ are column and row index, respectively, and $d,\mu$ are the number of dimensions and the quality of approximation.

# The Smolyak Grid in 2D: Tensor Table

![[@maliar-maliar] table 3: All Tensor Products](../assets/figs/smolyak-tensortab.png)



## Selecting Elements

* Denote the Smolyak grid for $d$ dimensions at level $\mu$ by $\mathcal{H}^{d,\mu}$.
* if $\mu=0$ we have $i_1+i_2\leq 2$. Only one point satisfies this, and 
	$$ \mathcal{H}^{2,0} = \{(0,0)\} $$
* if $\mu=1$ we have $i_1+i_2\leq 3$. Three cases satisfy this:
	1. $i_1 = 1, i_2=1 \rightarrow (0,0)$
	1. $i_1 = 1, i_2=2 \rightarrow (0,0),(0,-1),(0,1)$
	1. $i_1 = 2, i_2=1 \rightarrow (0,0),(-1,0),(1,0)$
	* Therefore, the unique elements from the union of all of those is
		$$ \mathcal{H}^{2,1} = \{(0,0),(-1,0),(1,0),(0,-1),(0,1)\} $$
* if $\mu=2$ we have $i_1+i_2\leq 4$. Six cases satisfy this:
	1. $i_1 = 1, i_2=1$
	1. $i_1 = 1, i_2=2$
	1. $i_1 = 2, i_2=1$
	1. $i_1 = 1, i_2=3$
	1. $i_1 = 2, i_2=2$
	1. $i_1 = 3, i_2=1$
	* Therefore, the unique elements from the union of all of those is
		$$ \mathcal{H}^{2,2} = \left\{(-1,1),(0,1),(1,1),(-1,0),(0,0),(1,0),(-1,-1),(0,-1),(1,-1),\left(\frac{-1}{\sqrt{2}},0\right),\left(\frac{1}{\sqrt{2}},0\right),\left(0,\frac{-1}{\sqrt{2}}\right),\left(0,\frac{1}{\sqrt{2}}\right)\right\} $$
* Note that those elements are on the diagonal from top left to bottom right expanding through all the tensor products on table 3.


## Size of Smolyak Grids

* The Smolyak grid grows much slower (at order $d$ to a power of $\mu$) than the Tensor grid (exponential growth)

![[@maliar-maliar] figure 2: Tensor vs Smolyak in 2D](../assets/figs/smolyak-vs-tensor.png)

![[@maliar-maliar] figure 4: Tensor vs Smolyak in 2D, number of grid points](../assets/figs/smolyak-tensor-points.png)



## Smolyak Polynomials

* Corresponding to the construction of grid points, there is the Smolyak way of constructing polynomials.
* This works exactly as before. We start with a one-dimensional set of basis functions (again Chebyshev here, again irrelevant):
	$$ \left\{1,x,2x^2-1,4x^3-3x,8x^4-8x^2+1\right\} $$
* Three nested sets:

$$ \begin{array}{l}
    i=1 : S_1 = \{1\} \\
    i=2 : S_2 = \{1,x,2x^2-1\} \\
    i=3 : S_3 = \left\{1,x,2x^2-1,4x^3-3x,8x^4-8x^2+1\right\}  \end{array} 
$$

* Denoting $\mathcal{P}^{d,\mu}$ the Smolyak polynomial, we follow exactly the same steps as for the grids to select elements of the full tensor product table 5:


![[@maliar-maliar] figure 5: All Smolyak Polynomials in 2D](../assets/figs/smolyak-polynomial.png)



## Smolyak Interpolation

This proceeds as in the previouses cases:

1. Evaluate $f$ at all grid points $\mathcal{H}^{d,\mu}$.
1. Evaluate the set of basis functions given by $\mathcal{P}^{d,\mu}$ at all grid points $\mathcal{H}^{d,\mu}$.
1. Solve for the interpolating coefficients by inverting the Basis function matrix.

## Extensions

* There is a lot of redundancy in computing the grids the way we did it.
* More sophisticated approaches take care not to compute repeated elements.

## Smolyak Grids in Julia

* There are at least 2 julia packages that implement this idea:
	* [https://github.com/EconForge/Smolyak](https://github.com/EconForge/Smolyak)  (can't find the function)
	* [https://github.com/alancrawford/Smolyak](https://github.com/alancrawford/Smolyak) (seems broken)




# Using `alancrawford/Smolyak`

From the unit tests of that package:

In [17]:
# Pkg.clone("https://github.com/alancrawford/Smolyak.git")
using Smolyak

# here is the true function
# in a real world application, evaluating this is the biggest cost
# we would like to evaluate only a few times.

slopes = rand(4)	
truefun(x) = 1.1 + slopes[1]*x[1] - slopes[2]*x[2]^2 + (slopes[3]*x[3])^3 * slopes[4] * x[4]

# choose approx level in each dim
mu = [1,2,2,1]
D = length(mu)
lb = -2*ones(length(mu))
ub = 12*ones(length(mu))
sg = SmolyakGrid(mu,lb,ub)
sb = SmolyakBasis(sg)
makeBasis!(sb)
sp = SmolyakPoly(sb)

for i in 1:sb.NumPts
    sp.Value[i] = truefun(sg.xGrid[i]) 	# Assign true fvals to poly
end
make_pinvBFt!(sp,sb)
Smolyak.makeCoef!(sp) 

# make basis on random point
NumObs = 50
X = Vector{Float64}[ Float64[lb[d]+( ub[d]- lb[d])*rand() for d in 1:D] for i in 1:NumObs]
sbX = SmolyakBasis(X,mu,lb,ub,0,0)
makeBasis!(sbX)
spX = SmolyakPoly(sbX)
copy!(spX.Coef,sp.Coef)   # assign precomputed coefs
ynew = makeValue!(spX,sbX) # Interpolated Values
valsnew = [truefun(i) for i in X]
scatter(ynew,valsnew,ylabel="truth",xlabel="approx")


LoadError: [91mArgumentError: Module Smolyak not found in current path.
Run `Pkg.add("Smolyak")` to install the Smolyak package.[39m